<a href="https://colab.research.google.com/github/WesleyVictors/python2/blob/main/3_14_ORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [104]:
### Escreva sua resposta aqui
import pandas as pd

data_professionals = pd.read_csv('salaries.csv')

data_professionals = data_professionals.rename(columns={
    "FIRST NAME": "first_name",
    "LAST NAME": "last_name",
    "SEX": "sex",
    "DOJ": "doj",
    "CURRENT DATE": "current_date",
    "DESIGNATION": "designation",
    "AGE": "age",
    "SALARY": "salary",
    "UNIT": "unit",
    "LEAVES USED": "leaves_used",
    "LEAVES REMAINING": "leaves_remaining",
    "RATINGS": "ratings",
    "PAST EXP": "past_exp"
})

display(data_professionals.head(5))




,first_name,last_name,sex,doj,current_date,designation,age,salary,unit,leaves_used,leaves_remaining,ratings,past_exp
0,TOMASA,ARMEN,F,5-18-2014,01-07-2016,Analyst,21.0,44570,Finance,24.0,6.0,2.0,0
1,ANNIE,NaN,F,NaN,01-07-2016,Associate,NaN,89207,Web,NaN,13.0,NaN,7
2,OLIVE,ANCY,F,7-28-2014,01-07-2016,Analyst,21.0,40955,Finance,23.0,7.0,3.0,0
3,CHERRY,AQUILAR,F,04-03-2013,01-07-2016,Analyst,22.0,45550,IT,22.0,8.0,3.0,0
4,LEON,ABOULAHOUD,M,11-20-2014,01-07-2016,Analyst,NaN,43161,Operations,27.0,3.0,NaN,3


#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [105]:
from datetime import date
from enum import Enum as PyEnum
from sqlalchemy import Enum, Float, Integer, String, Date, create_engine, text
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

class Base(DeclarativeBase):
    pass

class SexEnum(PyEnum):
    M = "M"
    F = "F"

class DesignationEnum(PyEnum):
    ANALYST = "Analyst"
    SENIOR_ANALYST = "Senior Analyst"
    SENIOR_MANAGER = "Senior Manager"
    MANAGER = "Manager"
    ASSOCIATE = "Associate"

class UnitEnum(PyEnum):
    IT = "IT"
    FINANCE = "Finance"
    MARKETING = "Marketing"
    WEB = "Web"
    OPERATIONS = "Operations"
    MANAGEMENT = "Management"

class DataProfessional(Base):
    __tablename__ = "data_professionals"

    id: Mapped[int]                        = mapped_column(primary_key=True, autoincrement=True)
    first_name: Mapped[str]                = mapped_column("first_name", String, nullable=True)
    last_name: Mapped[str]                 = mapped_column("last_name", String, nullable=True)
    sex: Mapped[SexEnum]                   = mapped_column("sex", Enum(SexEnum), nullable=True)
    doj: Mapped[date]                      = mapped_column("doj", Date, nullable=True)
    current_date: Mapped[date]             = mapped_column("current_date", Date, nullable=True)
    designation: Mapped[DesignationEnum] = mapped_column("designation",Enum(DesignationEnum, native_enum=False),nullable=True)
    #designation: Mapped[DesignationEnum]   = mapped_column("designation", Enum(DesignationEnum), nullable=True)
    age: Mapped[float]                     = mapped_column("age", nullable=True)
    salary: Mapped[float]                  = mapped_column("salary", nullable=True)
    unit: Mapped[UnitEnum]                 = mapped_column("unit", Enum(UnitEnum), nullable=True)
    leaves_used: Mapped[float]             = mapped_column("leaves_used", nullable=True)
    leaves_remaining: Mapped[float]        = mapped_column("leaves_remaining", nullable=True)
    ratings: Mapped[float]                 = mapped_column("ratings", nullable=True)
    past_exp: Mapped[float]                = mapped_column("past_exp", nullable=True)


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [106]:
engine = create_engine("sqlite:///salarios.db", echo=True)

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [108]:
Base.metadata.create_all(engine)

2025-06-30 16:09:42,638 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-06-30 16:09:42,642 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("data_professionals")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("data_professionals")


2025-06-30 16:09:42,644 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-06-30 16:09:42,646 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [109]:
data_professionals.to_sql("data_professionals", con=engine, if_exists="append", index=False)

2025-06-30 16:09:44,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-06-30 16:09:44,373 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("data_professionals")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("data_professionals")


2025-06-30 16:09:44,375 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2025-06-30 16:09:44,406 INFO sqlalchemy.engine.Engine [generated in 0.02064s] [('TOMASA', 'ARMEN', 'F', '5-18-2014', '01-07-2016', 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0), ('ANNIE', None, 'F', None, '01-07-2016', 'Associate', None, 89207, 'Web', None, 13.0, None, 7), ('OLIVE', 'ANCY', 'F', '7-28-2014', '01-07-2016', 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0), ('CHERRY', 'AQUILAR', 'F', '04-03-2013', '01-07-2016', 'Analyst', 22.0, 45550, 'IT', 22.0, 8.0, 3.0, 0), ('LEON', 'ABOULAHOUD', 'M', '11-20-2014', '01-07-2016', 'Analyst', None, 43161, 'Operations', 27.0, 3.0, None, 3), ('VICTORIA', None, 'F', '2-19-2013', '01-07-2016', 'Analyst', 22.0, 48736, 'Marketing', 20.0, 10.0, 4.0, 0), ('ELLIOT', 'AGULAR', 'M', '09-02-2013', '01-07-2016', 'Analyst', 22.0, 40339, 'Marketing', 19.0, 11.0, 5.0, 0), ('JACQUES', 'AKMAL', 'M', '12-05-2013', '01-07-2016', 'Analyst', None, 40058, 'Marketing', 29.0, 1.0, 2.0, 2)  ... displaying 10 of 2639 total bound parameter sets ...  ('T

INFO:sqlalchemy.engine.Engine:[generated in 0.02064s] [('TOMASA', 'ARMEN', 'F', '5-18-2014', '01-07-2016', 'Analyst', 21.0, 44570, 'Finance', 24.0, 6.0, 2.0, 0), ('ANNIE', None, 'F', None, '01-07-2016', 'Associate', None, 89207, 'Web', None, 13.0, None, 7), ('OLIVE', 'ANCY', 'F', '7-28-2014', '01-07-2016', 'Analyst', 21.0, 40955, 'Finance', 23.0, 7.0, 3.0, 0), ('CHERRY', 'AQUILAR', 'F', '04-03-2013', '01-07-2016', 'Analyst', 22.0, 45550, 'IT', 22.0, 8.0, 3.0, 0), ('LEON', 'ABOULAHOUD', 'M', '11-20-2014', '01-07-2016', 'Analyst', None, 43161, 'Operations', 27.0, 3.0, None, 3), ('VICTORIA', None, 'F', '2-19-2013', '01-07-2016', 'Analyst', 22.0, 48736, 'Marketing', 20.0, 10.0, 4.0, 0), ('ELLIOT', 'AGULAR', 'M', '09-02-2013', '01-07-2016', 'Analyst', 22.0, 40339, 'Marketing', 19.0, 11.0, 5.0, 0), ('JACQUES', 'AKMAL', 'M', '12-05-2013', '01-07-2016', 'Analyst', None, 40058, 'Marketing', 29.0, 1.0, 2.0, 2)  ... displaying 10 of 2639 total bound parameter sets ...  ('TERI', 'ANASTASIO', 'F', 

2025-06-30 16:09:44,419 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


2639

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [110]:
### Execute aqui sua query SQL com SQLAlchemy

query = """
SELECT
    designation,
    ROUND(MIN(salary) / 12, 2) AS min_monthly_salary,
    ROUND(MAX(salary) / 12, 2) AS max_monthly_salary,
    ROUND(AVG(salary) / 12, 2) AS avg_monthly_salary

FROM data_professionals
GROUP BY designation
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    for row in result:
        print(row)

2025-06-30 16:09:46,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-06-30 16:09:46,494 INFO sqlalchemy.engine.Engine 
SELECT 
    designation,
    ROUND(MIN(salary) / 12, 2) AS min_monthly_salary,
    ROUND(MAX(salary) / 12, 2) AS max_monthly_salary,
    ROUND(AVG(salary) / 12, 2) AS avg_monthly_salary
   
FROM data_professionals
GROUP BY designation



INFO:sqlalchemy.engine.Engine:
SELECT 
    designation,
    ROUND(MIN(salary) / 12, 2) AS min_monthly_salary,
    ROUND(MAX(salary) / 12, 2) AS max_monthly_salary,
    ROUND(AVG(salary) / 12, 2) AS avg_monthly_salary
   
FROM data_professionals
GROUP BY designation



2025-06-30 16:09:46,496 INFO sqlalchemy.engine.Engine [generated in 0.00357s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00357s] ()


('Analyst', 3333.42, 4165.0, 3751.68)
('Associate', 5846.17, 8300.25, 7266.92)
('Director', 17832.25, 32342.67, 23914.27)
('Manager', 8343.67, 12407.5, 10522.72)
('Senior Analyst', 4170.33, 5830.5, 4991.78)
('Senior Manager', 12614.42, 16631.42, 14888.69)
2025-06-30 16:09:46,501 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [111]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas

with engine.connect() as conn:
    df = pd.read_sql_query(query, conn)

df[["min_monthly_salary", "max_monthly_salary", "avg_monthly_salary"]] = df[
    ["min_monthly_salary", "max_monthly_salary", "avg_monthly_salary"]
].round(2)
print(df)


engine.dispose()

2025-06-30 16:09:49,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-06-30 16:09:49,026 INFO sqlalchemy.engine.Engine 
SELECT 
    designation,
    ROUND(MIN(salary) / 12, 2) AS min_monthly_salary,
    ROUND(MAX(salary) / 12, 2) AS max_monthly_salary,
    ROUND(AVG(salary) / 12, 2) AS avg_monthly_salary
   
FROM data_professionals
GROUP BY designation



INFO:sqlalchemy.engine.Engine:
SELECT 
    designation,
    ROUND(MIN(salary) / 12, 2) AS min_monthly_salary,
    ROUND(MAX(salary) / 12, 2) AS max_monthly_salary,
    ROUND(AVG(salary) / 12, 2) AS avg_monthly_salary
   
FROM data_professionals
GROUP BY designation



2025-06-30 16:09:49,028 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-06-30 16:09:49,033 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


      designation  min_monthly_salary  max_monthly_salary  avg_monthly_salary
0         Analyst             3333.42             4165.00             3751.68
1       Associate             5846.17             8300.25             7266.92
2        Director            17832.25            32342.67            23914.27
3         Manager             8343.67            12407.50            10522.72
4  Senior Analyst             4170.33             5830.50             4991.78
5  Senior Manager            12614.42            16631.42            14888.69


In [117]:
### Execute aqui sua query com SQLAlchemy ORM
from sqlalchemy import func, select, String
from sqlalchemy.orm import Session




#  esse trecho '.cast(String)' foi adicionado por que havia um conflito no mapeamento especificamente nessa consulta, então foi feito transformado a coluna "designation em uma strig dentro da consulta"
stmt = (
    select(
        DataProfessional.designation.cast(String),
        func.round(func.min(DataProfessional.salary) / 12, 2).label("min_monthly_salary"),
        func.round(func.max(DataProfessional.salary) / 12, 2).label("max_monthly_salary"),
        func.round(func.avg(DataProfessional.salary) / 12, 2).label("avg_monthly_salary"),
    )
    .group_by(DataProfessional.designation)
    .order_by(DataProfessional.designation)
)


with Session(engine) as session:
    res = session.execute(stmt).all()

print(f"{'DESIGNATION':<20} {'MIN':>10} {'MAX':>10} {'AVG':>10}")
print("-" * 52)
for designation, min_sal, max_sal, avg_sal in res:
    print(f"{designation:<20} {min_sal:>10.2f} {max_sal:>10.2f} {avg_sal:>10.2f}")

2025-06-30 16:23:28,305 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-06-30 16:23:28,308 INFO sqlalchemy.engine.Engine SELECT CAST(data_professionals.designation AS VARCHAR) AS designation, round(min(data_professionals.salary) / (? + 0.0), ?) AS min_monthly_salary, round(max(data_professionals.salary) / (? + 0.0), ?) AS max_monthly_salary, round(avg(data_professionals.salary) / (? + 0.0), ?) AS avg_monthly_salary 
FROM data_professionals GROUP BY data_professionals.designation ORDER BY data_professionals.designation


INFO:sqlalchemy.engine.Engine:SELECT CAST(data_professionals.designation AS VARCHAR) AS designation, round(min(data_professionals.salary) / (? + 0.0), ?) AS min_monthly_salary, round(max(data_professionals.salary) / (? + 0.0), ?) AS max_monthly_salary, round(avg(data_professionals.salary) / (? + 0.0), ?) AS avg_monthly_salary 
FROM data_professionals GROUP BY data_professionals.designation ORDER BY data_professionals.designation


2025-06-30 16:23:28,310 INFO sqlalchemy.engine.Engine [cached since 212.2s ago] (12, 2, 12, 2, 12, 2)


INFO:sqlalchemy.engine.Engine:[cached since 212.2s ago] (12, 2, 12, 2, 12, 2)


2025-06-30 16:23:28,316 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


DESIGNATION                 MIN        MAX        AVG
----------------------------------------------------
Analyst                 3333.42    4165.00    3751.68
Associate               5846.17    8300.25    7266.92
Director               17832.25   32342.67   23914.27
Manager                 8343.67   12407.50   10522.72
Senior Analyst          4170.33    5830.50    4991.78
Senior Manager         12614.42   16631.42   14888.69
